In [1]:
import pickle

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import jax
import jax.numpy as jnp

import optax
from OTPE import OSTL, OTTT, OTPE, Approx_OTPE, OSTL_mod, OTPE_mod

from jax.tree_util import Partial, tree_map, tree_leaves, tree_structure, tree_unflatten
import spiking_learning as sl

import randman_dataset as rd
import numpy as np
from utils import gen_test_data, cos_sim_train_func, online_sim_train_func, custom_snn, bp_snn


In [2]:
output_size = 10
nlayers = 3
dim = 3
seq_len = 20
manifold_seed_val = 0
init_seed_val = 0
manifold_seed = jax.random.PRNGKey(manifold_seed_val)
init_seed = jax.random.split(jax.random.PRNGKey(init_seed_val))[0]
dtype = jnp.float32#jnp.bfloat16
tau = dtype(3.)
batch_sz = 128
spike_fn = sl.fast_sigmoid
n_iter = 1000 # 2000
layer_name = 128
update_time = 'online'
timing = 'time'
if timing=='time':
    t_name = 'time'
    t = True
elif timing == 'rate':
    t_name = 'rate'
    t = False
if layer_name == 128:
    layer_sz = lambda i: 128
elif layer_name == 512:
    layer_sz = lambda i: 512
elif layer_name == 'double':
    layer_sz = lambda i: min(10*2**i,240)

optimizer = optax.adamax(dtype(0.001))

2023-08-28 20:24:32.208121: W external/xla/xla/service/gpu/nvptx_compiler.cc:564] The NVIDIA driver's CUDA version is 12.1 which is older than the ptxas CUDA version (12.2.91). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [3]:
train_data,train_labels = rd.make_spiking_dataset(nb_classes=10, nb_units=50, nb_steps=seq_len, nb_samples=1000, dim_manifold=dim, alpha=1., nb_spikes=1, seed=manifold_seed,seed2=manifold_seed,shuffle=False,dtype=dtype)

In [4]:
gen_data = Partial(rd.make_spiking_dataset,nb_classes=10, nb_units=50, nb_steps=seq_len, nb_samples=1000, dim_manifold=dim, alpha=1., nb_spikes=1, seed=manifold_seed,shuffle=True,time_encode=t,dtype=dtype)

In [5]:
test_data,test_labels = gen_test_data(gen_data,1,manifold_seed)

/afs/crc.nd.edu/user/t/tsumme/miniconda3/envs/snn/lib/python3.11/site-packages/jax/_src/random.py:440: FutureWarning: jax.random.shuffle is deprecated and will be removed in a future release. Use jax.random.permutation with independent=True.
  warnings.warn(msg, FutureWarning)


In [6]:
carry = [OTPE.initialize_carry(dtype=dtype)]*nlayers

In [7]:
OTTTmodel = custom_snn(output_sz=output_size, n_layers=nlayers, mod1=OTTT, mod2=OTTT, spike_fn=spike_fn, layer_sz=layer_sz, dtype=dtype)
OSTLmodel = custom_snn(output_sz=output_size, n_layers=nlayers, mod1=OSTL, mod2=OSTL, spike_fn=spike_fn, layer_sz=layer_sz, dtype=dtype)
OTPEmodel = custom_snn(output_sz=output_size, n_layers=nlayers, mod1=OSTL_mod, mod2=OTPE_mod, spike_fn=spike_fn, layer_sz=layer_sz, dtype=dtype)
Approx_OTPEmodel = custom_snn(output_sz=output_size, n_layers=nlayers, mod1=OTTT, mod2=Approx_OTPE, spike_fn=spike_fn, layer_sz=layer_sz, dtype=dtype)
carry = [OTPE.initialize_carry(dtype=dtype)]*nlayers
params = OTPEmodel.init(init_seed,carry,train_data[0,:batch_sz])
carry,s = OTPEmodel.apply(params,carry,train_data[0,:batch_sz])
opt_state = optimizer.init(params)
orig_params = params

In [8]:
test_carry = [OTPE.test_carry()]*nlayers
test_carry,_ = OTPEmodel.apply(params,test_carry,train_data[0])

In [9]:
bp_model = bp_snn(output_sz=output_size, n_layers=nlayers, spike_fn=spike_fn, layer_sz=layer_sz, dtype=dtype)
bp_carry = carry
bp_params = bp_model.init(init_seed,bp_carry,train_data[0,:batch_sz])
struct = tree_structure(bp_params)
bp_params = tree_unflatten(struct,tree_leaves(orig_params))

bp_carry,s = bp_model.apply(bp_params,bp_carry,train_data[0,:batch_sz])
bp_opt_state = optimizer.init(bp_params)

In [10]:
carry = tree_map(lambda x: jnp.zeros_like(x,dtype),carry)
test_carry = tree_map(lambda x: jnp.zeros_like(x,dtype),test_carry)

In [11]:
key = jax.random.split(init_seed)[0]
cos = []
cos_per = []
val_acc = []
train_loss = []
all_params = [params]*4
all_params.append(bp_params)
all_opt = [opt_state]*4
all_opt.append(bp_opt_state)
carry = tree_map(lambda x: jnp.zeros_like(x,dtype),carry)
test_carry = tree_map(lambda x: jnp.zeros_like(x,dtype),test_carry)
best_acc = 0
best_params = [0]

In [12]:
offline_training = jax.jit(Partial(cos_sim_train_func,OTTTmodel,
                       Approx_OTPEmodel,
                       OSTLmodel,
                       OTPEmodel,
                       bp_model,
                       optimizer,
                       carry,
                       test_carry,
                       test_data,
                       test_labels,
                       batch_sz,
                       gen_data
                       ))

In [13]:
online_training = jax.jit(Partial(online_sim_train_func,OTTTmodel,
                       Approx_OTPEmodel,
                       OSTLmodel,
                       OTPEmodel,
                       optimizer,
                       carry,
                       test_carry,
                       test_data,
                       test_labels,
                       batch_sz,
                       gen_data
                       ))

In [14]:
# with open('randman_data/models/model_{}layer_{}_{}dim_{}_{}seqlen_{}iter_{}manifold_{}seed_{}_sub_fs_adamax'.format(nlayers,layer_name,dim,update_time,seq_len,0,manifold_seed_val,init_seed_val,t_name),'wb') as file:
#             pickle.dump(tree_map(jnp.float32,all_params),file,protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
for epoch in range(n_iter):        
    
    if update_time == 'offline':

        all_loss, all_cosines, all_cosines_per, all_acc, all_params, all_opt, key = offline_training(all_params,all_opt,key)
        

        cos.append(np.stack(list(tree_map(jnp.float32,all_cosines))))
        cos_per.append(np.stack(list((tree_map(jnp.float32,all_cosines_per)))))
    
    elif update_time == 'online':

        all_loss, all_acc, all_params, all_opt, key = online_training(all_params,all_opt,key)


    val_acc.append(np.stack(list(tree_map(jnp.float32,all_acc))))

    train_loss.append(np.stack(list(tree_map(jnp.float32,all_loss))))

    print(epoch)
    print(val_acc[-1])
    #print(cos[-1])
    #print(cos_per[-1])
    #print(all_cosines_per)

    # if (epoch+1)%200 == 0: #200
    #     with open('randman_data/models/model_{}layer_{}_{}dim_{}_{}seqlen_{}iter_{}manifold_{}seed_{}_sub_fs_adamax'.format(nlayers,layer_name,dim,update_time,seq_len,epoch+1,manifold_seed_val,init_seed_val,t_name),'wb') as file:
    #         pickle.dump(tree_map(jnp.float32,all_params),file,protocol=pickle.HIGHEST_PROTOCOL)

0
[0. 0. 0. 0.]
1
[0.     0.0081 0.     0.    ]
2
[0.     0.0705 0.     0.0275]
3
[4.6000001e-03 4.7599997e-02 9.9999997e-05 2.9700000e-02]
4
[0.0438 0.0122 0.01   0.0271]
5
[0.0351 0.0055 0.0169 0.0176]
6
[0.0022 0.0341 0.0008 0.0579]
7
[0.0217 0.0438 0.0013 0.0867]
8
[0.0669     0.0517     0.0103     0.07759999]
9
[0.0284 0.0861 0.0126 0.0725]
10
[0.0348     0.0947     0.0565     0.13159999]
11
[0.11099999 0.0988     0.12199999 0.1874    ]
12
[0.1335     0.1287     0.13509999 0.1988    ]
13
[0.12709999 0.1293     0.1432     0.20019999]
14
[0.1258 0.1469 0.1398 0.221 ]
15
[0.1437     0.2399     0.14909999 0.21769999]
16
[0.1706 0.278  0.1758 0.2547]
17
[0.1704     0.19839999 0.174      0.22389999]
18
[0.18679999 0.1698     0.1829     0.26929998]
19
[0.2074 0.2395 0.1892 0.3258]
20
[0.20709999 0.2886     0.19299999 0.34939998]
21
[0.21239999 0.296      0.2082     0.3624    ]
22
[0.22549999 0.2827     0.2166     0.3421    ]
23
[0.2411     0.2744     0.22819999 0.3469    ]
24
[0.24159999

In [16]:
np.save('randman_data/layer_cosine_similarity/sim_{}layer_{}_{}dim_{}_{}seqlen_{}iter_{}manifold_{}_sub_fs_adamax_{}seed'.format(nlayers,layer_name,dim,update_time,seq_len,n_iter,manifold_seed_val,t_name,init_seed_val),cos_per)
np.save('randman_data/model_cosine_similarity/sim_{}layer_{}_{}dim_{}_{}seqlen_{}iter_{}manifold_{}_sub_fs_adamax_{}seed'.format(nlayers,layer_name,dim,update_time,seq_len,n_iter,manifold_seed_val,t_name,init_seed_val),cos)
np.save('randman_data/accuracy/sim_{}layer_{}_{}dim_{}_{}seqlen_{}iter_{}manifold_{}_sub_fs_adamax_{}seed'.format(nlayers,layer_name,dim,update_time,seq_len,n_iter,manifold_seed_val,t_name,init_seed_val),val_acc)
np.save('randman_data/loss/sim_{}layer_{}_{}dim_{}_{}seqlen_{}iter_{}manifold_{}_sub_fs_adamax_{}seed'.format(nlayers,layer_name,dim,update_time,seq_len,n_iter,manifold_seed_val,t_name,init_seed_val),train_loss)